
# Multi-objective irrigation optimization
_____
### Mikhail Gasanov 

In [4]:
from experiments.crop_model_en import Irrigation

class Optimization(Irrigation):
    def __init__(self):
        super().__init__()
        self.year=None
        self.optimal_dates_irrigation = None
        self.num_process = 12
        self.optimizer_counter = 0
        self.container_of_mean_yields = []
        self.container_of_mean_water_loss = []
        self.container_of_irrigation_amount = []
        self.irrigation_dates_for_many_years_optim = None
        

    def multiobjective(self, x):
        """
        Minimize multiobjective function to define 
        best dates and ammounts of water for 20 years
        """
        x_dates = x[:len(self.user_parameters['irrigation_events'])]
        x_ammounts = x[len(self.user_parameters['irrigation_events']):]
        amounts = [float(i) for i in x_ammounts]


        inputs_years = np.arange(self.NASA_start_year, self.NASA_start_year+20)

        self.irrigation_dates_for_many_years_optim = self.irrigation_dates(x_dates)
        crop_sim_for_20_years = []
        water_loss_for_20_years = []
        for year in inputs_years:
            #change year from json-year to historical
            self.date_crop_start = self.year_changer(self.user_parameters['crop_start'],year)
            self.date_crop_end = self.year_changer(self.user_parameters['crop_end'],year)
            #convet dates from int to dt.datetime
            dates_irrigation = self.irrigation_dates(x_dates)
            #Setup irrigation ammount
    
            dates_irrigation = [self.year_changer(obj, year) for obj in dates_irrigation]
            dates_npk, npk_list = self.user_parameters['npk_events'], self.user_parameters['npk']
            dates_npk = [self.year_changer(obj, year) for obj in dates_npk]
            agromanagement = self.agromanager_writer(self.user_parameters['crop_name'], dates_irrigation, dates_npk, amounts, npk_list)
            self.load_model()
            self.run_simulation_manager(agromanagement)
            output = pd.DataFrame(self.output).set_index("day")
            
            ## append to list loss
            crop_sim_for_20_years.append(output)
            water_loss_for_20_years.append(self.total_ammount_of_losed_water)
            
        #select only last day crop yield 
        yield_of_crop_sim_for_20_years = [(crop_sim_for_20_years[i]['TWSO'][-1]/1000) for i in range(len(crop_sim_for_20_years))]
        # calculate mean
        out_yield = np.mean(yield_of_crop_sim_for_20_years)
        out_water_loss = np.mean(water_loss_for_20_years)
        irrigation_sum = np.sum(amounts)
        self.container_of_irrigation_amount.append(irrigation_sum)
        self.container_of_mean_water_loss.append(out_water_loss)
        self.container_of_mean_yields.append(out_yield)
        
        #counter for optimizer
        self.optimizer_counter += 1
        return -out_yield, out_water_loss ## check this in out_yield -- MAX, out_water -- MIN !!!!
    
    


Platform not recognized, using system temp directory for PCSE settings.
Platform not recognized, using system temp directory for PCSE settings.


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import json
import os
import argparse
import multiprocessing
import matplotlib.pyplot as plt

from pymoo.model.problem import Problem
from pymoo.model.callback import Callback
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover


from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_crossover, get_mutation, get_sampling
from pymoo.optimize import minimize


path_to_data_dir  = './util/input_data/' 
path_to_user_file = './util/input_data/malino_potato.json' 
path_to_CSV_weather = './data/meteo'
path_to_npy_files = './experiments/test/potato/'


# NSGA-II parameters: generation size and popultion size

num_generation = 5 

population_size = 20



WOFOST = Optimization()
# Load user data: geo, crop, num of irriagtion activities from JSON file
with open(path_to_user_file, 'r') as f:
    WOFOST.user_parameters = json.load(f)

# Round geodata to load from NASA POWER
def round_geoposition(x, prec=1, base=.5):
    return round(base * round(float(x)/base),prec)
latitude = round_geoposition(WOFOST.user_parameters['latitude'])
longitude = round_geoposition(WOFOST.user_parameters['longitude'])

# Crop selected by user
crop_name = WOFOST.user_parameters['crop_name']

#load historical weather data
path_CSV_dir = path_to_CSV_weather
WOFOST.weather_loader(path_CSV_dir, latitude, longitude)
WOFOST.data_dir = path_to_data_dir

crop_results=[]

# Mask for optimizer: here optimizer search for optimal dates for irrigation and optimal amount of water
# Dates in int format and amounts of water in float format (cm)

mask = ['int']*len(WOFOST.user_parameters['irrigation_events']) + ['real']*len(WOFOST.user_parameters['irrigation_ammounts'])

# NSGA-II settings
sampling = MixedVariableSampling(mask, {
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random")
})

crossover = MixedVariableCrossover(mask, {
    "real": get_crossover("real_sbx", prob=1.0, eta=3.0),
    "int": get_crossover("int_sbx", prob=1.0, eta=3.0)
})

mutation = MixedVariableMutation(mask, {
    "real": get_mutation("real_pm", eta=3.0),
    "int": get_mutation("int_pm", eta=3.0)
})

# Variable to set upper limit for search of date - vegetation season in range 
#from sowing date (0) and harvesting date (max_number_of_days)

max_number_of_days = len(pd.date_range(start=WOFOST.user_parameters['crop_start'],end=WOFOST.user_parameters['crop_end']))

# Variables to setup range of irrigation water, we selested from 1 cm to 15 cm of water
x_lower = ([1]*len(WOFOST.user_parameters['irrigation_events'])+[1]*len(WOFOST.user_parameters['irrigation_ammounts']))
x_upper = ([max_number_of_days-5]*len(WOFOST.user_parameters['irrigation_events'])+[15]*len(WOFOST.user_parameters['irrigation_ammounts']))
num_of_var = len(x_lower)


# Create problem to minimize in PyMOO
class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=num_of_var, n_obj=2, 
                        xl=x_lower, 
                        xu=x_upper,
                        elementwise_evaluation=True)


    def _evaluate(self, x, out, *args, **kwargs):
        
        # finction to minimize - f1 and f2 loss
        f1, f2 = WOFOST.multiobjective(x)

        out['F'] = [f1,f2]

# PyMOO settings which data to save 
class MyCallback(Callback):

    def __init__(self) -> None:
        super().__init__()
        self.data["best"] = []

    def notify(self, algorithm):
        self.data["best"].append(algorithm.pop.get("F").min())


problem = MyProblem()
# Define algorithm - we select NSGA-II, PyMOO package allows to utilize another
algorithm = NSGA2(
    pop_size=population_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True,
)

# Start optimization
print('Start search for optimal solution!')
res = minimize(
    problem,
    algorithm,
    ('n_gen', num_generation),
    seed=1,
    callback=MyCallback(),
    verbose=True,
    save_history=True
)



# Save data for plots 


import copy
mdt = copy.deepcopy(res.F)
mdt[:,0]=mdt[:,0]*-1

# Save data for plots 
path_to_folder = path_to_npy_files

if os.path.isdir(path_to_folder)==False:
    os.mkdir(path_to_folder)
    
# Save important info for plots
np.save(path_to_folder + WOFOST.user_parameters['crop_name']+'_irrigation_ammount', WOFOST.container_of_irrigation_amount)
np.save(path_to_folder + WOFOST.user_parameters['crop_name']+'_water_loss', WOFOST.container_of_mean_water_loss)
np.save(path_to_folder + WOFOST.user_parameters['crop_name']+'_crop_yields', WOFOST.container_of_mean_yields)
np.save(path_to_folder + WOFOST.user_parameters['crop_name']+'_function_values_for_paretto', mdt)
np.save(path_to_folder + WOFOST.user_parameters['crop_name']+'_optimal_solutions', res.X)




print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

LOAD FROM LOCAL CSV WEATHER DATABASE
Start search for optimal solution!
n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |      20 |       6 |            - |            -
    2 |      40 |      10 |  0.014932169 |            f
    3 |      60 |      11 |  0.115564511 |        ideal
